## Más sobre funciones y entornos

### Alcance léxico 

En la sección anterior, utilizamos el mecanismo `copiar-poner-modificar`. Los ejemplos demuestran dos casos en los que este mecanismo sucede. Cuando un objeto tiene varios nombres o se pasa como un argumento a una función, modificarlo produce que el objeto se copie y es la versión copiada la que realmente se modifica.

Para modificar un objeto fuera de una función, se utilizó el operador  `<<-`, que encuentra la variable fuera de la función primero y modifica ese objeto en lugar de copiar uno localmente. Esto lleva a una idea importante de que una función tiene dentro y fuera. Dentro de una función, podemos referirnos a variables y funciones externas.

Por ejemplo, la siguiente función utiliza dos variables externas:

In [1]:
primer_numero <- 1
final_numero <- 10
fun1 <- function(x) {
    c(primer_numero, x, final_numero)
}

Primero creamos dos variables y definimos una función llamada `fun1`. La función coloca `primer_numero`, el  argumento `x` y `final_numero` en un nuevo vector. Es claro que `primer_numero` y  `final_numero` no están definidos en la función sino fuera de ella, mientras que `x` es el argumento de la función. 

Veamos si funciona:

In [2]:
fun1(c(4, 5, 6))

[1]  1  4  5  6 10

La función trabaja, consiguiendo  el valor de las dos variables fuera de la función. Podemos adivinar que cuando definimos la función, los valores se capturan de manera que  `primer_numero `y `final_numero` en `fun1` tomen los valores externos. De hecho, dos experimentos pueden ser realizados para demostrar que es incorrecto.

El primer experimento es simple. Vamos a eliminar las dos variables:

In [3]:
rm(primer_numero, final_numero)
fun1(c(4, 5, 6))

ERROR: Error in fun1(c(4, 5, 6)): objeto 'primer_numero' no encontrado


Entonces, la función ya no funciona. Si los valores de las dos variables se capturan cuando se define la función, la eliminación de ellas no debe detener la función. 

El segundo experimento es al revés. Vamos a eliminar la función, así como las dos variables. Primero definiremos la función:

In [4]:
rm(fun1, primer_numero, final_numero)
fun1 <- function(x) {
    c(primer_numero, x, final_numero)
}


Warning message in rm(fun1, primer_numero, final_numero):
“objeto 'primer_numero' no encontrado”Warning message in rm(fun1, primer_numero, final_numero):
“objeto 'final_numero' no encontrado”

Si la creación de la función tiene que capturar las dos variables que no están presentes en él, el código anterior debería dar lugar a un error diciendo `primer_numero` y `fina_numero`  no son encontrados. 

Claramente, no hay error y la función se ha creado correctamente. Vamos a llamarla ahora:

In [5]:
fun1(c(4, 5, 6))

ERROR: Error in fun1(c(4, 5, 6)): objeto 'primer_numero' no encontrado


La función no funciona porque las dos variables no se encuentran. A continuación, definiremos las dos variables y llamaremos de nuevo la función con el mismo argumento:

In [6]:
primer_numero <- 1
final_numero <- 10
fun1(c(4, 5, 6))

[1]  1  4  5  6 10

La función funciona de nuevo. Esto lleva a la conclusión de que la función realmente trata de buscar las variables cuando se llama. En realidad, durante la ejecución de la función, cuando se encuentra un símbolo, primero lo buscará dentro de la función. Más específicamente, si el símbolo es pasado  como un argumento o se crea dentro de la función, el símbolo se determinará  y se utilizará su valor.


Supongamos que primero creamos una variable `p` y luego definimos una función `fun2` en la que otra variable `p` se crea y se usa en el valor a devolver:

In [7]:
p <- 0
fun2 <- function(x) {
    p <- 1
    x + p
}

Cuando llamamos a la función, ¿cuál `p`, la función  `fun2` se utilizará en `x + p`?. Vamos a averiguarlo;

In [8]:
fun2(1)

[1] 2

La salida deja claro que `x + p` usa `p` definido dentro de la función. El flujo es simple. Primero, `p <- 1` crea una nueva variable `p` con valor `1` en lugar de cambiar `p` fuera de la función. Entonces, se evalúa `x + p`, con `x` siendo determinado  como el argumento pasado y `p` como la variable local que acaba de definirse. La regla es que sólo si una variable no está presente dentro de la función se buscará fuera.

Sin embargo, ¿qué significa exactamente **fuera**? La pregunta es más sutil de lo que parece ser. Supongamos que creamos las dos funciones siguientes:

In [9]:
f1 <- function(x) {
    x + p
}
g1 <- function(x) {
    p <- 1
    f1(x)
}

La primera función `f1` simplemente agrega dos variables: `x` es un argumento y `p` es una variable que aún no se encuentra fuera. La segunda función `g1` define una variable `p` dentro y llama a `f1`. La pregunta es, ¿`f1` encontrará  `p` dentro de `g1` cuando se llama a `g1`?

In [10]:
g1(0)

[1] 0

Desafortunadamente, `f1` no puede encontrar `p` dentro de `g1`, incluso si  `f1` se llama en `g1`. Si definimos `p` y luego llamamos `g1` de nuevo, la función funciona:

In [11]:
p <- 1
g1(0)

[1] 1

Lo que hizo a `g1` funcionar es que cuando `f1` se llama y `p` no se puede encontrar dentro de `f1`, buscará donde se define `f1` en lugar de donde se llama. Este mecanismo se denomina **alcance léxico**. En el código anterior, definimos `p` en el mismo alcance  donde se define `f1`. Entonces, `f1` puede encontrar a `p` cuando se llama dentro de `g1`.

La misma regla de alcance también se aplica a cómo  `<<-` encuentra variables. Por ejemplo, el siguiente código define una variable `m` y dos funciones, `f2` y `g2`, en el mismo alcance. En `f2`, `m` se establece en `2`. Sin embargo, en `g2`, se define una variable  local `m`   y entonces se llama a `f2`:

In [12]:
m <- 1
f2 <- function(x) {
    m <<- 2
    x
}
g2 <- function(x) {
    m <- 1
    f2(x)
    cat(sprintf("[g2] m: %d\n", m))
}

Tan pronto como se llama a `f2`, se imprime el valor de `m` en `g2`. Vamos a llamar a `g2` y ver qué pasa:

In [13]:
g2(1)

[g2] m: 1


El texto  muestra que el valor de `m` en `g2` permanece sin cambios, pero el valor de `m` fuera de `f2` y `g2` se cambia como se puede verificar:

In [14]:
m

[1] 2

El experimento anterior confirma que `m <<- 2` sigue la regla de alcance léxico.

Los siguientes dos ejemplos parecen aún más complejos. Las funciones están anidadas. En `h`, no sólo creamos variables locales como `p` y `q` sino también una función local `f3`, en la que se define otra variable `p` local:

In [15]:
h <- function(x) {
    p <- 1
    q <- 2
    cat(sprintf("1. [f1] p: %d, q: %d\n", p, q))
    f3 <- function(x) {
        p <- 3
        cat(sprintf("2. [f2] p: %d, q: %d\n", p, q))
        c(x = x, p = p, q = q)
    }
    cat(sprintf("3. [f1] p: %d, q: %d\n", p, q))
    f3(x)
}

Por el alcance léxico, se puede  predecir el resultado dado una entrada arbitraria `x`. Añadimos algunas funciones `cat()` para facilitar el seguimiento de los valores de las variables en cada nivel de alcance. El mensaje `cat()` incluye un orden, el alcance de la función y los valores de `p` y `q`. Ahora, vamos a ejecutar `f(0)` y  mostrar el resultado:

In [16]:
h(0)

1. [f1] p: 1, q: 2
3. [f1] p: 1, q: 2
2. [f2] p: 3, q: 2


x p q 
0 3 2

El orden de ejecución de las tres funciones `cat()` son `1`, `3` y `2` y los valores de `p` y `q` en cada alcance son consistentes con las reglas de alcance léxicas. En el siguiente ejemplo, también usaremos `<<-`:

In [17]:
r <- function(x) {
    p <- 1
    q <- 2
    cat(sprintf("1. [f1] p: %d, q: %d\n", p, q))
    g3 <- function(x) {
        p <<- 3
        p <- 2
        cat(sprintf("2. [f2] p: %d, q: %d\n", p, q))
        c(x = x, p = p, q = q)
    }
    cat(sprintf("3. [f1] p: %d, q: %d\n", p, q))
    resultado <- g3(x)
    cat(sprintf("4. [f1] p: %d, q: %d\n", p, q))
    resultado
}

Se puede analizar el flujo de la función mediante la predicción del orden de ejecución y los valores de las variables impresas:

In [18]:
r(0)

1. [f1] p: 1, q: 2
3. [f1] p: 1, q: 2
2. [f2] p: 2, q: 2
4. [f1] p: 3, q: 2


x p q 
0 2 2

El siguiente ejemplo ilustra el principio más básico del alcance léxico y no debería haber ningún problema en predecir la salida.

In [19]:
f4 <- function() {
  x <- 1
  y <- 2
  c(x, y)
}
f4()
rm(f4)

[1] 1 2

Si un nombre no está definido dentro de una función, `R` buscará en un nivel más alto.

In [20]:
x <- 2
g4 <- function() {
  y <- 1
  c(x, y)
}
g4()
rm(x, g4)

[1] 2 1

Las mismas reglas se aplican si una función se define dentro de otra función: miramos dentro de la función actual, entonces donde esa función fue definida y así sucesivamente, todo el camino hasta el entorno global y luego a otros paquetes cargados.

In [21]:
x <- 1
h1 <- function() {
  y <- 2
  i <- function() {
    z <- 3
    c(x, y, z)
  }
  i()
}
h1()
rm(x, h1)

[1] 1 2 3

Las mismas reglas se aplican a **clausuras**, funciones creadas por otras funciones. Aquí solo veremos cómo interactúan con el alcance. La siguiente función, `j()`, devuelve una función. 

Esto parece un poco mágico (cómo `R` sabe cuál es el valor de `y` después de que la función ha sido llamada). Funciona porque `k` preserva el entorno en el que fue definido y porque el entorno incluye el valor de `y`. 

In [22]:
j <- function(x) {
    y <- 2
    function() {
        c(x, y)
    }
}
k <- j(1)
k()
rm(j, k)

[1] 1 2

Los mismos principios se aplican independientemente del tipo de valor asociado, encontrar funciones trabaja exactamente de la misma manera que encontrar variables:

In [23]:
l <- function(x) x + 1
m <- function() {
  l <- function(x) x * 2
  l(10)
}
m()

[1] 20

In [24]:
rm(l, m)

Para las funciones, hay un pequeño ajuste a la regla. Si se está utilizando un nombre en un contexto en el que es obvio que se desea una función (por ejemplo, `f(3)`), `R` ignorará los objetos que no son funciones mientras está buscando. En el ejemplo siguiente `n` toma un valor diferente dependiendo de si `R` está buscando una función o una variable.

In [25]:
n <- function(x) x / 2
o <- function() {
  n <- 4
  n(n)
}
o()

[1] 2

Sin embargo, usar el mismo nombre para funciones y otros objetos hará confuso el código y generalmente es mejor evitarlo.

In [26]:
rm(n, o)

¿Qué sucede con los valores entre invocaciones de una función? ¿Qué pasará la primera vez que se ejecute esta función? ¿Qué pasará la segunda vez? (Si no has visto `exists()` antes: devuelve `TRUE` si hay una variable de ese nombre, de lo contrario devuelve `FALSE`.)

In [27]:
j1 <- function() {
  if (!exists("a")) {
    a <- 1
  } else {
    a <- a + 1
  }
  a
}
j1()
rm(j1)

[1] 1

Se puede estar ser sorprendido por que devuelve el mismo valor `1`, cada vez. Esto se debe a que cada vez que se llama una función, se crea un nuevo entorno para la ejecución. Una función no tiene forma de saber qué pasó la última vez que se ejecutó. Cada invocación es completamente independiente. 

El alcance léxico determina dónde buscar valores, no cuándo buscarlos. `R` busca valores cuando se ejecuta la función, no cuando se crea. Esto significa que la salida de una función puede ser diferente dependiendo de los objetos fuera de su entorno:

In [28]:
f5 <- function() x
x <- 15
f5()

[1] 15

In [29]:
x <- 20
f5()

[1] 20

Por lo general,  se desea evitar este comportamiento porque significa que la función ya no es autónoma. Este es un error común - si se  comete un error ortográfico en el  código, no se obtendrá un error al crear la función  y es posible que ni siquiera obtenga un error cuando se ejecute la función, dependiendo de qué variables se definen en el entorno global.

Una forma de detectar este problema es la función `findGlobals()` de `codetools`. Esta función lista todas las dependencias externas de una función:

In [30]:
library("codetools")

f6 <- function() x + 1
codetools::findGlobals(f6)

[1] "+" "x"

Otra forma de tratar de resolver el problema sería cambiar manualmente el entorno de la función a `emptyenv()`, un entorno que no contiene absolutamente nada:

In [31]:
environment(f6) <- emptyenv()
f6()

ERROR: Error in f6(): no se pudo encontrar la función "+"


Esto no funciona porque `R` se basa en el alcance léxico para encontrar todo, incluso el operador `+`. Nunca es posible hacer una función completamente autónoma porque siempre se debe confiar en las funciones definidas en el paquete  base  o en  otros paquetes.

Podemos  usar esta misma idea para hacer otras cosas que son extremadamente desacertadas. Por ejemplo, puesto que todos los operadores estándar en `R` son funciones, puede anularlas con sus propias alternativas. 

### Algo de entornos en R

Un entorno es un tipo muy especial de objeto en `R`, pero se utiliza en todas partes desde la implementación de una llamada de función hasta el  mecanismo de alcance léxico. De hecho, cuando ejecuta una parte de código `R`, se ejecuta en un determinado entorno. Para saber en qué entorno estamos ejecutando el código, podemos llamar a `environment()`:

In [32]:
environment()

<environment: R_GlobalEnv>

Un entorno es también un objeto  que podemos crear y trabajar. Por ejemplo, podemos asignar el entorno actual a una variable y crear nuevos símbolos en este entorno:

In [33]:
global <- environment()
global$algun_objeto <- 1

La asignación anterior es equivalente a llamar directamente a `algun_objeto <- 1`, pues este objeto se encuentra en  el entorno global. Mientras se ejecute el código anterior, se modifica el entorno global y `algun_objeto` obtiene un valor:

In [34]:
algun_objeto

[1] 1

Hay otras maneras de acceder al entorno global. Por ejemplo, `globalenv()` y `.GlobalEnv` se refieren al entorno global:

In [35]:
globalenv()

<environment: R_GlobalEnv>

In [36]:
.GlobalEnv

<environment: R_GlobalEnv>

El entorno global (`globalenv()`) es el espacio de trabajo del usuario, mientras que el entorno base (`baseenv()`) proporciona funciones y operadores básicos:

In [37]:
baseenv()

<environment: base>

El entorno global y el entorno base son los entornos incorporados de `R` más importantes. Ahora, nos podemos preguntar ¿Cuál es el entorno padre del entorno global? ¿Y el entorno base? ¿ y sus abuelos?.

La siguiente función se puede utilizar para averiguar la cadena de un entorno dado:

In [38]:
pariente_entorno <- function(env) {
    while (TRUE) {
        nombre <- environmentName(env)
        txt <- if (nzchar(nombre)) nombre else format(env)
            cat(txt, "\n")
            env <- parent.env(env)
            }
}

In [39]:
pariente_entorno(globalenv())

R_GlobalEnv 
package:codetools 
jupyter:irkernel 
jupyter:irkernel 
package:stats 
package:graphics 
package:grDevices 
package:utils 
package:datasets 
package:methods 
Autoloads 
base 
R_EmptyEnv 


ERROR: Error in parent.env(env): el entorno vacio no esta asociado


Se debe tener  en cuenta que la cadena termina en un entorno denominado **entorno vacio **, que es el único entorno que no tiene nada y no tiene un entorno padre. También hay una función `emptyenv()` que hace referencia al entorno vacío, pero  `parent.env(emptyenv())` provocará un error. Esto explica por qué los `pariente_entorno()` siempre terminan con un error.

La cadena de entornos es una combinación de entornos integrados y entornos de paquetes. Podemos llamar a `search()`, para obtener la ruta de búsqueda del símbolo en la perspectiva del entorno global:

In [40]:
search()

[1] ".GlobalEnv"        "package:codetools" "jupyter:irkernel" 
 [4] "jupyter:irkernel"  "package:stats"     "package:graphics" 
 [7] "package:grDevices" "package:utils"     "package:datasets" 
[10] "package:methods"   "Autoloads"         "package:base"

Dado el conocimiento de la búsqueda de símbolos a lo largo de una cadena de entornos, podemos determinar el proceso en detalle de cómo se evalúa el siguiente código en el entorno global:

In [41]:
median(c(1, 2, 1 + 3))

[1] 2

La expresión parece simple, pero su proceso de evaluación es más complejo de lo que parece. Primero, busca `median` a lo largo de la cadena. Se encuentra en el entorno del paquete `stats`. Luego, se busca `c` que se encuentra en el entorno base. Finalmente, puede ser sorprendido cuando se  busca `+` (también es una función!), que se encuentra en el entorno base.

De hecho, cada vez que  se adjunta un paquete, el entorno del paquete se insertará antes que el entorno global en la ruta de búsqueda. Si dos paquetes exportan funciones con nombres en conflicto, las funciones definidas en el paquete posteriormente enmascararán las anteriormente definidas  ya que se convierten en un padre más cercano al entorno global.

### Entornos asociados con funciones 

Los entornos gobiernan la búsqueda de símbolos no sólo en el nivel global, sino también en el nivel de función. Hay tres entornos importantes asociados con las funciones y su proceso de ejecución: el entorno de ejecución, el entorno de cierre y el entorno de llamada.

Cada vez que se llama una función, se crea un nuevo entorno llamado para alojar el proceso de ejecución. Este es el entorno de ejecución de la llamada de función. Los argumentos de la función y las variables que creamos en la función son en realidad las variables en el entorno de ejecución.

Al igual que todos los demás entornos, el entorno de ejecución de una función se crea con un entorno padre. Ese entorno padre, también llamado el entorno de cierre de la función, es el entorno donde se define la función. Esto significa que durante la ejecución de la función, cualquier variable que no esté definida en el entorno de ejecución será buscada en el entorno de cierre. Esto es exactamente lo que hace posible el alcance léxico.

A veces también es útil conocer el entorno de llamada, es decir, el entorno en el que se llama a la función.  Podemos utilizar `parent.frame()` para obtener el entorno de llamada de la función que se está ejecutando actualmente.

Para demostrar estos conceptos, supongamos que definimos la siguiente función:

In [42]:
entorno_funcion <- function() {
    cat("Entorno de ejecucion: ")
    print(environment())
    cat("Entorno de cierre: ")
    print(parent.env(environment()))
}

Esta  función no hace nada, pero imprime los entornos de ejecución y de cierre cuando se le llama:

In [43]:
entorno_funcion()

Entorno de ejecucion: <environment: 0x30f7850>
Entorno de cierre: <environment: R_GlobalEnv>


In [44]:
entorno_funcion()

Entorno de ejecucion: <environment: 0x305c1e8>
Entorno de cierre: <environment: R_GlobalEnv>


In [45]:
entorno_funcion()

Entorno de ejecucion: <environment: 0x2fa73e0>
Entorno de cierre: <environment: R_GlobalEnv>


Tenga en cuenta que cada vez que se llama a la función, el entorno de ejecución es diferente, pero el entorno de cierre, sigue siendo el mismo. De hecho, cuando se define la función, se determina el entorno de cierre. Podemos llamar a `environment()` sobre una función para obtener su entorno de cierre:

In [46]:
environment(entorno_funcion)

<environment: R_GlobalEnv>

El ejemplo siguiente implica los tres entornos de tres funciones anidadas. En cada función, se imprimen el entorno de ejecución, el entorno de cierre y el entorno de llamada. 

In [47]:
f7 <- function() {
    cat("[f7] Ejecutando en ")
    print(environment())
    cat("[f7] Cerrado por ")
    print(parent.env(environment()))
    cat("[f7] Llamado desde ")
    print(parent.frame())
    f8 <- function() {
        cat("[f8] Ejecutando en ")
        print(environment())
        cat("[f8] Cerrado por ")
        print(parent.env(environment()))
        cat("[f8] Llamando desde ")
        print(parent.frame())
    }
    f9 <- function() {
        cat("[f9] Ejecutando en ")
        print(environment())
        cat("[f9] Cerrado por ")
        print(parent.env(environment()))
        cat("[f9] Llamado desde")
        print(parent.frame())
        f8()
    }
    f9()
}

Llamamos a `f7` y averiguamos cuándo se imprime cada mensaje. La salida requiere un cierto esfuerzo para leer en su forma original. Dividimos la salida en partes para facilitar la lectura mientras se preserva el orden de la salida para la consistencia.

Tenga en cuenta que los entornos temporalmente creados sólo tienen direcciones de memoria (por ejemplo, `0x0000000016a39fe8`) en lugar de un nombre común como el entorno global (`R_GlobalEnv`).


```
f7()
[f7] Ejecutando en <environment: 0x2b4e4e8>
[f7] Cerrado por <environment: R_GlobalEnv>
[f7] Llamado desde <environment: R_GlobalEnv>
```

Cuando llamamos `f7`, sus entornos asociados se imprimen como se supone y luego se definen `f8` y `f9`  y finalmente se llama `f9`, lo cual continúa produciendo el siguiente resultado de texto:

```
[f9] Ejecutando en <environment: 0x2b527d0>
[f9] Cerrado por <environment: 0x2b4e4e8>
[f9] Llamado desde<environment: 0x2b4e4e8>
```

Entonces, `f8` se llama en `f9`, lo que produce  el siguiente texto:

```
[f8] Ejecutando en <environment: 0x2b56e38>
[f8] Cerrado por <environment: 0x2b4e4e8>
[f8] Llamando desde <environment: 0x2b527d0>
```

Los mensajes impresos muestran los siguientes hechos:

* Tanto el entorno de cierre como el entorno de llamada de `f7` son el entorno global.

* El entorno de cierre y el entorno de llamada de `f9`, así como el entorno de cierre de `f8`, son los entornos de ejecución de `f7`.

* El entorno de llamada de `f8` es el entorno de ejecución de `f9`.

Los hechos anteriores son consistentes con los siguientes hechos:

* `f7` se define y se llama en el entorno global.
* `f9` se define y se llama en `f7`.
* `f8` se define en `f7` pero se llama en `f9`.

### Lecturas


* [Environments in R](http://digitheadslabnotebook.blogspot.pe/2011/06/environments-in-r.html).
* [A Tutorial on Using Functions in R!](https://www.datacamp.com/community/tutorials/functions-in-r-a-tutorial).